In [1]:
%pylab inline
from JSAnimation import IPython_display
from matplotlib import animation
from __future__ import print_function
import time
from channels import *
from compartment import *
from membrane import *
from params import *

from csdmodel1d import *

import numpy as np
np.seterr(all='ignore')
from scipy.integrate import ode

Populating the interactive namespace from numpy and matplotlib


In [2]:
model = CSDModelInterval(N=10,dx=50e-6) # define the model, grid spacing is 100 microns, or approximately two cell widths

# Define the compartments, and the membranes
ecs = Compartment("ecs")
neuron = CellCompartment("neuron",density = 2e5) # 2e5 neurons per meter, 4e10 per sq meter
glia = CellCompartment("glia",density = 2e5) #2e5 glia per meter

neuronal_er = CellCompartment("neuron_er",density = 2e5)
glial_er = CellCompartment("glia_er",density = 2e5)

neuronal_mito = CellCompartment("neuron_mito",density = 2e5)
glial_mito = CellCompartment("glial_mito",density = 2e5)

neuron_mem = Membrane("neuronal",inside=neuron,outside=ecs,Cm=Cn,phi_m=-70e-3)
glial_mem = Membrane("glial",inside=glia,outside=ecs,Cm=Cg,phi_m=-85e-3)

neuronal_er_mem = Membrane("neuronal_er",inside=neuronal_er,outside=neuron,Cm=Cn,phi_m=0.0)
neuronal_er_mem = Membrane("glial_er",inside=glial_er,outside=glia,Cm=Cn,phi_m=0.0)


# Add the compartments to the model
model.addCompartment(ecs,fraction=0.2) # ECS take 20% of the total volume
model.addCompartment(neuron,fraction=0.4-0.04) # Neurons take up 40% of the total volume
model.addCompartment(glia,fraction=0.4-0.04) # Neurons take up 40% of the total volume
model.addCompartment(neuronal_er,fraction = 0.02)
model.addCompartment(glial_er,fraction = 0.02)
model.addCompartment(neuronal_mito,fraction = 0.02)
model.addCompartment(glial_mito,fraction = 0.02)

# Add ion species
ecs.addSpecies(K,Ke0,name='K_e')
ecs.addSpecies(Cl,Cle0,name='Cl_e')
ecs.addSpecies(Na,Nae0,name='Na_e')
ecs.addSpecies(Ca,Cae0,name='Ca_e')
ecs.addSpecies(Glu,ge0,name = "g_e") # 4 micromolar in ecs

neuron.addSpecies(K,Ki0,0,'K_n')
neuron.addSpecies(Na,Nai0,0,'Na_n')
neuron.addSpecies(Cl,Cli0,0,'Cl_n')
neuron.addSpecies(Ca,Cai0,0,'Ca_n')
neuron.addSpecies(Glu,1e-6,name = "g_n")


glia.addSpecies(K,Ki0,name='K_g')
glia.addSpecies(Na,Nai0,name='Na_g')
glia.addSpecies(Cl,Cli0,name='Cl_g')
glia.addSpecies(Ca,Cai0,0,'Ca_g')

# add channels
neuron_mem.addChannel(NaTChannel(),10000.) # 10000 per neuron?
neuron_mem.addChannel(NaPChannel(),100.) # 100 per neuron
neuron_mem.addChannel(KDRChannel(),10000.) # number of channels per neuron
neuron_mem.addChannel(KAChannel(),10000.) # number of channels per neuron
#neuron_mem.addChannel(CaPChannel(),10000.) # number of channels per neuron
#neuron_mem.addChannel(CaLChannel(),10000.) # number of channels per neuron
#neuron_mem.addChannel(CaNChannel(),10000.) # number of channels per neuron

#neuron_mem.addChannel(KIRChannel(),10000.)
#neuron_mem.addChannel(gNMDAChannel(),10000.)

#neuron_mem.addChannel(PMCAPump(),10000) # PMCA pump
#neuron_mem.addChannel(NaCaExchangePump(),1000) # sodium-calcium exchanger
neuron_mem.addChannel(NaKATPasePump(),10000) # 5000 ATPase per neuron
neuron_mem.addChannel(NonSpecificChlorideChannel(-0.07),100000)
neuron_mem.addChannel(AquaPorin(),1e-1) # Add water exchange

glial_mem.addChannel(KIRChannel(),50000) # KIR Channel
glial_mem.addChannel(NaKATPasePump(),50000) # 10000000 ATPase per glia
#glial_mem.addChannel(PMCAPump(),10000)
glial_mem.addChannel(NonSpecificChlorideChannel(-0.085),100000)
glial_mem.addChannel(AquaPorin(),1e-5) # Add water exchange

#glial_mem.addChannel(CaPChannel(),10000.) # number of channels per neuron
#glial_mem.addChannel(CaLChannel(),10000.) # number of channels per neuron
#glial_mem.addChannel(CaNChannel(),10000.) # number of channels per neuron


In [3]:
model.addMembrane(neuron_mem)
model.addMembrane(glial_mem)

neuron_mem.addLeakChannels()
neuron.balanceWith(ecs)
glial_mem.addLeakChannels()
glia.balanceWith(ecs)

model.assembleSystem()

system_state = model.getInternalVars()

Ion: Na+, P_leak: 1.37E-10
Ion: K+, P_leak: 1.47E-09
Ion: Na+, P_leak: 5.82E-10
Ion: K+, P_leak: 2.66E-08
Adding a leak channel does nothing to balance Cl-


In [4]:
'''
#Hole method for initiation - very slow!!
neuron_hole = HoleChannel([K,Na,Cl],1e-6)
density = np.zeros(model.N)
density[0:3] = 10
neuron_mem.addChannel(neuron_hole,density)

glial_hole = HoleChannel([K,Na,Cl],1e-6)
glial_mem.addChannel(glial_hole,density)
'''

ecs.values[K][0] += 0.020
ecs.values[Cl][0] += 0.020

y = model.getInternalVars()
model.odesolver.set_initial_value(y,0)

t_old = 0
model.odesolver.t = 0
dt1 = 0 # timescale separation
t_old1 = 0
system_states = []
t = []
meanflux = 0.0
t.append(0)
start_time = time.time()

In [ ]:
print('{:<8} {:<11} {:<11} {:<11} {:<11} {:<11} {:<11} {:<11}'.format('time', 'V_n','V_g', 'K_e', 'K_n','K_g','Cl_e','Ca_n') )
print('{:<8.3f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f}'.format(model.odesolver.t, 1e3*neuron_mem.phi()[0], 1e3*glial_mem.phi()[0], 1e3*ecs.value(K)[0], 1e3*neuron.value(K)[0], 1e3*glia.value(K)[0], 1e3*ecs.value(Cl)[0], 1e3*neuron.value(Ca)[0]), end='\r')

try:
    y = model.odesolver.integrate(model.odesolver.t+1e-6)
    system_states.append(y)
    t.append(model.odesolver.t)
    while model.odesolver.successful() and model.odesolver.t < 240.0:
        y = model.odesolver.integrate(model.odesolver.t+1e-3)
        if sum(isnan(y))>0: break
        print('{:<8.3f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f}'.format(model.odesolver.t, 1e3*neuron_mem.phi(y)[0], 1e3*glial_mem.phi(y)[0], 1e3*ecs.value(K,y)[0], 1e3*neuron.value(K,y)[0], 1e3*glia.value(K,y)[0], 1e3*ecs.value(Cl,y)[0], 1e3*neuron.value(Ca,y)[0]), end='\r')
        system_states.append(y)
        t.append(model.odesolver.t)
except KeyboardInterrupt:
    print('{:<8.3f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f}'.format(model.odesolver.t, 1e3*neuron_mem.phi(y)[0], 1e3*glial_mem.phi(y)[0], 1e3*ecs.value(K,y)[0], 1e3*neuron.value(K,y)[0], 1e3*glia.value(K,y)[0], 1e3*ecs.value(Cl,y)[0], 1e3*neuron.value(Ca,y)[0]))
   
print('{:<8.3f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f}'.format(model.odesolver.t, 1e3*neuron_mem.phi(y)[0], 1e3*glial_mem.phi(y)[0], 1e3*ecs.value(K,y)[0], 1e3*neuron.value(K,y)[0], 1e3*glia.value(K,y)[0], 1e3*ecs.value(Cl,y)[0],1e3*neuron.value(Ca,y)[0]), end='\r')
elapsed_time = time.time() - start_time
print('{:<8.3f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f} {:<11.7f}'.format(model.odesolver.t, 1e3*neuron_mem.phi(y)[0], 1e3*glial_mem.phi(y)[0], 1e3*ecs.value(K,y)[0], 1e3*neuron.value(K,y)[0], 1e3*glia.value(K,y)[0], 1e3*ecs.value(Cl,y)[0], 1e3*neuron.value(Ca,y)[0]))
print("\nElapsed time %.3f" % (elapsed_time) )

time     V_n         V_g         K_e         K_n         K_g         Cl_e        Ca_n       
0.008    -37.2434173 -48.5615036 23.3696481  133.5000902 133.5028484 165.3704759 0.0000010  

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, model.N*model.dx*1e6), ylim=(3.5e-3, 30e-3))
line, = ax.plot([], [], lw=2)
ttl = ax.text(120, .025, '')

dt = 50

def init():
    line.set_data([], [])
    ttl.set_text('t='+str(0.0))
    return line,

def animate(i):
    line.set_data(model.x*1e6,ecs.value(K,system_states[i*dt]))
    ttl.set_text('t='+str(t[i*dt]))
    return line,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=1000, interval=100, blit=True)

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, 500), ylim=(133.4e-3, 133.6e-3))
line, = ax.plot([], [], lw=2)

def init():
    line.set_data([], [])
    return line,

def animate(i):
    line.set_data(model.x*1e6,neuron.value(K,system_states[i]))
    return line,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=200, interval=100, blit=True)

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, model.N*model.dx*1e6), ylim=(0,2e-3))
line, = ax.plot([], [], lw=2)
ttl = ax.text(120, .025, '')

dt = 100

def init():
    line.set_data([], [])
    ttl.set_text('t='+str(0.0))
    return line,

def animate(i):
    line.set_data(model.x*1e6,1e3*neuron.value(Ca,system_states[i*dt]))
    ttl.set_text('t='+str(t[i*dt]))
    return line,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=1000, interval=100, blit=True)

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, model.N*model.dx*1e6), ylim=(0,2e-3))
line, = ax.plot([], [], lw=2)
ttl = ax.text(120, .025, '')

dt = 100

def init():
    line.set_data([], [])
    ttl.set_text('t='+str(0.0))
    return line,

def animate(i):
    line.set_data(model.x*1e6,1e3*glia.value(Ca,system_states[i*dt]))
    ttl.set_text('t='+str(t[i*dt]))
    return line,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=1000, interval=100, blit=True)

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, model.N*model.dx*1e6), ylim=(-75,30))
line, = ax.plot([], [], lw=2)
ttl = ax.text(120, .025, '')

dt = 100

def init():
    line.set_data([], [])
    ttl.set_text('t='+str(0.0))
    return line,

def animate(i):
    line.set_data(model.x*1e6,1e3*neuron_mem.phi(system_states[i*dt]))
    ttl.set_text('t='+str(t[i*dt]))
    return line,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=1000, interval=100, blit=True)

In [ ]:
fig = plt.figure()
ax = plt.axes(xlim=(0, model.N*model.dx*1e6), ylim=(0.05,0.95))
line, = ax.plot([], [], lw=2)
ttl = ax.text(120, .055, '')

dt = 100

def init():
    line.set_data([], [])
    ttl.set_text('t='+str(0.0))
    return line,

def animate(i):
    line.set_data(model.x*1e6,model.volumefraction(ecs,system_states[i*dt]))
    ttl.set_text('t='+str(t[i*dt]))
    return line,

animation.FuncAnimation(fig, animate, init_func=init,
                        frames=1000, interval=100, blit=True)

In [ ]:
neuron_mem.waterFlow(system_states[10])

In [ ]:
model.volumefraction(ecs,system_states[100])

In [ ]:
model.volumefractions(system_states[100])